# MasterCamp_TD_Project

### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

### Importation des données

In [18]:
# Chargement des données
df = 'data/valeursfoncieres-2023.txt'

# Lecture du fichier par échantillons de 10 000 lignes
df_chunks = pd.read_csv(df, sep='|', chunksize=10000, decimal=',')

# Concaténation des échantillons
df = pd.concat(df_chunks)

# Affichage des 5 premières lignes
df.head()

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,2.0,Appartement,NaN,233.0,8.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2023,Vente,...,NaN,0,1.0,Maison,NaN,64.0,3.0,S,NaN,988.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,0,1.0,Maison,NaN,73.0,3.0,S,NaN,835.0


In [19]:
# On garde les colonnes qui nous intéressent : 

# show les noms des colonnes :
df.columns
"""Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')"""

# On garde les colonnes qui nous intéressent : "No disposition", "Date mutation", "Nature mutation", "Valeur fonciere", "Code departement", "Code commune", "Prefixe de section", "Section", "No plan", "Code type local", "Surface reelle bati", "Surface terrain"
df = df[["No disposition", "Date mutation", "Nature mutation", "Valeur fonciere", "Code departement", "Code commune", "Prefixe de section", "Section", "No plan", "Code type local", "Surface reelle bati", "Surface terrain", "Nombre pieces principales", "Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]]
df.tail()




,No disposition,Date mutation,Nature mutation,Valeur fonciere,Code departement,Code commune,Prefixe de section,Section,No plan,Code type local,Surface reelle bati,Surface terrain,Nombre pieces principales,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot
3727000,1,05/10/2023,Adjudication,3931000.0,75,107,NaN,BU,4,3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3727001,1,05/10/2023,Adjudication,3931000.0,75,107,NaN,BU,4,3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3727002,1,05/10/2023,Adjudication,3931000.0,75,107,NaN,BU,4,3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3727003,1,28/12/2023,Vente,98250.0,75,115,NaN,FV,100,3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3727004,1,28/12/2023,Vente,98250.0,75,115,NaN,FV,100,2.0,18.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
